In [1]:
import io

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import numpy as np
import scipy.stats as stats


from docx import Document

In [2]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
    converter.close()
    fake_file_handle.close()
    
    if text:
        return text

def var_in_pdf_length(filename = 'Data/DLW.pdf', v = 9):
    s = extract_text_from_pdf(filename)
    n0 = s.find('Вариант ' + str(v))
    n1 = s.find('Вариант ' + str(v+1))
    if n0 == -1:
        n0 = 0
    if n1 == -1:
        n1 = len(s)
    space0 = 'Вариант '+str(v)+'     '
    space1 = '               '
    #print(len(space0),len(space1))
    w = s[n0+len(space0):n1-len(space1)]
    w = w.replace(',', '.')
    w = w.split(' ')
    l = [[],[],[]]
    for i in range(len(w)):
        if i%3 == 0:
            l[0].append(float(w[i]))
        if (i-1)%3 == 0:
            l[1].append(float(w[i]))
        if (i-2)%3 == 0:
            l[2].append(float(w[i]))
    return l

ql = var_in_pdf_length()
ql = [np.array(i) for i in ql]
for i in ql:
    print(i)

print()
N = ql[0].size
m = len(ql)
print(N,m)
#for i in ql:
#    print(i.size)

[ 1.23711  0.8985  -0.62379 -1.99397  2.31921 -1.13083 -4.48306 -1.88237
  1.23293  0.696   -5.98482 -1.84592 -2.20889 -1.89087 -2.12074 -1.73964
 -0.17203  1.58969  4.38888 -2.68677]
[ 0.85209  2.6936  -3.10639  3.14631 -3.63701  1.0854  -2.62187 -0.95353
  2.31578 -0.28418 -1.49992 -0.01161 -3.69538  0.07593  1.07141 -3.80871
 -1.47559  2.34929 -0.26896  0.74149]
[-0.64836 -1.31322 -1.79283 -0.47805 -2.49926 -0.82844 -3.46558  1.2153
  3.29139 -2.80116  3.94803 -0.36222  4.76533  2.09255  0.28011  2.80244
  1.4327   0.56925 -2.25504 -0.     ]

20 3


In [3]:
def mean1(x):
    return sum(x)/x.size
def mean2(x):
    return sum(x**2)/x.size

for i in range(m):
    print('mean1('+str(i+1)+') =',mean1(ql[i]),'   mean2('+str(i+1)+') =',mean2(ql[i]))
print()

def S2(x):
    return (mean2(x)-mean1(x)**2)*x.size/(x.size-1)

for i in range(m):
    print('S2('+str(i+1)+') =',S2(ql[i]))
print()

def TNM(x,y):
    o1 = mean1(x)-mean1(y)
    o2 = S2(x)*(x.size-1) + S2(y)*(y.size-1)
    o3 = x.size*y.size*(x.size+y.size-2)/(x.size+y.size)
    return o1*((o3/o2)**0.5)

for k in range(m):
    for i in range(k):
        print('TNM('+str(i+1)+';'+str(k+1)+') =',TNM(ql[i],ql[k]))

mean1(1) = -0.8200689999999999    mean2(1) = 6.162381582740001
mean1(2) = -0.3515925000000001    mean2(2) = 4.752388836444999
mean1(3) = 0.19764700000000004    mean2(3) = 5.144326894760001

S2(1) = 5.7788088610305275
S2(2) = 4.872391105672367
S2(3) = 5.373960587527369

TNM(1;2) = -0.6419534528695119
TNM(1;3) = -1.3628568391296252
TNM(2;3) = -0.7673475869434438


In [4]:
def u_all(ql):
    s = 0
    n = 0
    for i in ql:
        n += i.size
        s += i.sum()
    return s/n

def u_one(x):
    return x.sum()/x.size

def s_all(ql):
    s = 0
    u = u_all(ql)
    for i in ql:
        for k in i:
            s += (u - k)**2
    return s

def s_fakt(ql):
    s = 0
    u = u_all(ql)
    for i in ql:
        s += i.size*(u-u_one(i))**2
    return s

def s_ost(ql):
    return s_all(ql) - s_fakt(ql)

def FNM_1(ql):
    sf2 = s_fakt(ql)/(len(ql)-1)
    so2 = s_ost(ql)/((len(ql)-1)*ql[0].size)
    return sf2/so2

print('u_all = ', u_all(ql))
for i in range(len(ql)):
    print('u_'+str(i+1)+' = ',u_one(ql[i]))
print('s_all = ', s_all(ql))
print('s_fakt = ', s_fakt(ql))
print('s_ost = ', s_ost(ql))
print('FNM_1 = ',FNM_1(ql))

u_all =  -0.3246715
u_1 =  -0.8200689999999998
u_2 =  -0.3515925000000001
u_3 =  0.19764699999999996
s_all =  314.85725130416495
s_fakt =  10.379200773789993
s_ost =  304.47805053037496
FNM_1 =  0.6817700491520032


In [5]:
def Sm1(x,y):
    return max(S2(x),S2(y))

def Sm2(x,y):
    return min(S2(x),S2(y))

def FNM_2(x,y):
    return Sm1(x,y)/Sm2(x,y)

def k1(x,y):
    if Sm1(x,y) == S2(x):
        return x.size-1
    if Sm1(x,y) == S2(y):
        return y.size-1

def k2(x,y):
    if Sm2(x,y) == S2(x):
        return x.size-1
    if Sm2(x,y) == S2(y):
        return y.size-1

for k in range(3):
    for i in range(k):
        print('Sm1('+str(i+1)+';'+str(k+1)+') = ', Sm1(ql[i],ql[k]))
        print('Sm2('+str(i+1)+';'+str(k+1)+') = ', Sm2(ql[i],ql[k]))
        print('FNM_2('+str(i+1)+';'+str(k+1)+') = ', FNM_2(ql[i],ql[k]))
        print('k1('+str(i+1)+';'+str(k+1)+') = ',k1(ql[i],ql[k]),'     k2('+str(i+1)+';'+str(k+1)+') = ',k2(ql[i],ql[k]))
        print()

Sm1(1;2) =  5.7788088610305275
Sm2(1;2) =  4.872391105672367
FNM_2(1;2) =  1.1860314034115451
k1(1;2) =  19      k2(1;2) =  19

Sm1(1;3) =  5.7788088610305275
Sm2(1;3) =  5.373960587527369
FNM_2(1;3) =  1.075335177269961
k1(1;3) =  19      k2(1;3) =  19

Sm1(2;3) =  5.373960587527369
Sm2(2;3) =  4.872391105672367
FNM_2(2;3) =  1.1029411373136861
k1(2;3) =  19      k2(2;3) =  19



In [6]:
print('N1')
print('3)','ANOVA(1,2,3) = ', stats.f_oneway(ql[0],ql[1],ql[2]).pvalue)
for k in range(3):
    for i in range(k):
        print('4)','t_test_2(' + str(i+1) + ';'+ str(k+1)+') = ', stats.ttest_ind(ql[i],ql[k]).pvalue)
for k in range(3):
    for i in range(k):
        print('5)','welch_test(' + str(i+1) + ';'+ str(k+1)+') = ', stats.ttest_ind(ql[i],ql[k], equal_var = False).pvalue)
print()
print('N2')
print('2)','bartlett_test(1,2,3) = ',stats.bartlett(ql[0],ql[1],ql[2]).pvalue)
for k in range(3):
    for i in range(k):
        print('3)','var_test(' + str(i+1) + ';'+ str(k+1)+') = ', stats.bartlett(ql[i],ql[k]).pvalue)

N1
3) ANOVA(1,2,3) =  0.38468502548678885
4) t_test_2(1;2) =  0.5247573549424901
4) t_test_2(1;3) =  0.1809497953516925
4) t_test_2(2;3) =  0.44761670674601195
5) welch_test(1;2) =  0.5247851287049549
5) welch_test(1;3) =  0.18096029486012624
5) welch_test(2;3) =  0.4476280064656546

N2
2) bartlett_test(1,2,3) =  0.9345180276555175
3) var_test(1;2) =  0.7137521351726992
3) var_test(1;3) =  0.8758447617893016
3) var_test(2;3) =  0.8330857713105322


In [7]:
head_1_1_str = ['столбцы', 'x', 'y', 'x^2', 'y^2', 'S_x^2', 'S_y^2','T_NM']
def out_1_1(ql):
    out = []
    for i in range(len(ql)):
        for k in range(i+1,len(ql)):
            l = [
                '('+str(i+1)+','+str(k+1)+')',
                mean1(ql[i]),
                mean1(ql[k]),
                mean2(ql[i]),
                mean2(ql[k]),
                S2(ql[i]),
                S2(ql[k]),
                TNM(ql[i],ql[k])
            ]
            out.append(l)
    return out


head_1_2_str = ['S_общ', 'S_факт', 'S_ост', 'S_факт^2', 'S_ост^2', 'k_1', 'k_2', 'F_NM']
def out_1_2(ql):
    sf2 = s_fakt(ql)/(len(ql)-1)
    so2 = s_ost(ql)/((len(ql)-1)*ql[0].size)
    k1 = len(ql)
    k2 = 0
    for i in ql:
        k2 += (i.size-1)
    return [
        s_all(ql), s_fakt(ql), s_ost(ql), sf2, so2, k1, k2, FNM_1(ql)
    ]


def out_1_3(ql):
    return stats.f_oneway(ql[0],ql[1],ql[2]).pvalue


head_1_4_str = ['столбцы','pval[t_test_2]']
def out_1_4(ql):
    l = []
    for i in range(3):
        for k in range(i):
            l.append(['('+str(k+1)+','+str(i+1)+')',stats.ttest_ind(ql[k],ql[i]).pvalue])
    return l


head_1_5_str = ['столбцы','pval[welch_test]']
def out_1_5(ql):
    l = []
    for i in range(3):
        for k in range(i):
            l.append(['('+str(k+1)+','+str(i+1)+')',stats.ttest_ind(ql[k],ql[i],equal_var = False).pvalue])
    return l


head_2_1_str = ['столбцы', 'S_1^2','S_2^2','k_1','k_2','F_NM']
def out_2_1(ql):
    out = []
    for i in range(3):
        for k in range(i):
            x = k
            y = i
            l = [
                '('+str(x+1)+'; '+str(y+1)+')',
                Sm1(ql[x],ql[y]),
                Sm2(ql[x],ql[y]),
                k1(ql[x],ql[y]),
                k2(ql[x],ql[y]),
                FNM_2(ql[x],ql[y])
            ]
            out.append(l)
    return out


def out_2_2(ql):
    return stats.bartlett(ql[0],ql[1],ql[2]).pvalue


head_2_3_str = ['столбцы', 'pval[var_test]']
def out_2_3(ql):
    out = []
    for i in range(3):
        for k in range(i):
            x = k
            y = i
            l = [
                '('+str(x+1)+'; '+str(y+1)+')',
                stats.bartlett(ql[x],ql[y]).pvalue
            ]
            out.append(l)
    return out


In [8]:
expert = {
    True : 'верно',
    False : 'не верно'
}


anal_head_1_1_str = ['столбцы', '|T_NN|','t_кр,alpha(2N-2)','вывод']
def anal_1_1(ql):
    ao = []
    N = ql[0].size
    for k in range(3):
        for i in range(k):
            l = [
                '('+str(i+1)+'; '+str(k+1)+')',
                abs(TNM(ql[i],ql[k])),
                stats.t.ppf(0.97,2*N-2),
                expert[abs(TNM(ql[i],ql[k])) <= stats.t.ppf(0.97,2*N-2)]
            ]
            ao.append(l)
    return ao

anal_head_1_2_str = ['F_NM', 'alpha', 'z_alpha', 'вывод']
def anal_1_2(ql):
    m = len(ql)-1
    n = len(ql)*(ql[0].size-1)
    f = FNM_1(ql)
    z = stats.f.ppf(0.94,m,n)
    ao = [f, '0.06', z, expert[f <= z]]
    return [ao]


anal_head_1_3_str = ['pval[anova]', 'alpha', 'вывод']
def anal_1_3(ql):
    anova = stats.f_oneway(ql[0],ql[1],ql[2]).pvalue
    return [[anova, '0.06', expert[anova >= 0.06]]]


anal_head_1_4_str = ['столбцы','pval [t_test_2]', 'alpha', 'вывод']
def anal_1_4(ql):
    o_1_4 = out_1_4(ql)
    ao = []
    for i in o_1_4:
        l = [i[0],i[1],'0.06',expert[i[1] >= 0.06]]
        ao.append(l)
    return ao

anal_head_1_5_str = ['столбцы','pval [welch_test]', 'alpha', 'вывод']
def anal_1_5(ql):
    o_1_5 = out_1_5(ql)
    ao = []
    for i in o_1_5:
        l = [i[0],i[1],'0.06',expert[i[1] >= 0.06]]
        ao.append(l)
    return ao


anal_head_2_1_str = ['столбцы', 'F_NM','F(alpha/2,k_1,k_2)','вывод']
def anal_2_1(ql):
    z = stats.f.ppf(0.97,N-1,N-1)
    o_2_1 = out_2_1(ql)
    ao = []
    for i in o_2_1:
        l = [i[0], i[5], z, expert[i[5] <= z]]
        ao.append(l)
    return ao


anal_head_2_2_str = ['pval[bartlett_test]', 'alpha', 'вывод']
def anal_2_2(ql, alpha = 0.06):
    pv = out_2_2(ql)
    return [[pv, '0.06', expert[pv >= 0.06]]]


anal_head_2_3_str = ['строка','pval[bartlett_test]', 'alpha', 'вывод']
def anal_2_3(ql, alpha = 0.06):
    o_2_3 = out_2_3(ql)
    ao = []
    for i in o_2_3:
        l = [i[0], i[1], '0.06', expert[i[1] >=0.06]]
        ao.append(l)
    return ao

In [9]:
print('N1')
print(anal_1_1(ql))
print(anal_1_2(ql))
print(anal_1_3(ql))
print(anal_1_4(ql))
print(anal_1_5(ql))
print('N2')
print(anal_2_1(ql))
print(anal_2_2(ql))
print(anal_2_3(ql))

N1
[['(1; 2)', 0.6419534528695119, 1.938632713539906, 'верно'], ['(1; 3)', 1.3628568391296252, 1.938632713539906, 'верно'], ['(2; 3)', 0.7673475869434438, 1.938632713539906, 'верно']]
[[0.6817700491520032, '0.06', 2.9569596997732552, 'верно']]
[[0.38468502548678885, '0.06', 'верно']]
[['(1,2)', 0.5247573549424901, '0.06', 'верно'], ['(1,3)', 0.1809497953516925, '0.06', 'верно'], ['(2,3)', 0.44761670674601195, '0.06', 'верно']]
[['(1,2)', 0.5247851287049549, '0.06', 'верно'], ['(1,3)', 0.18096029486012624, '0.06', 'верно'], ['(2,3)', 0.4476280064656546, '0.06', 'верно']]
N2
[['(1; 2)', 1.1860314034115451, 2.4307190858231333, 'верно'], ['(1; 3)', 1.075335177269961, 2.4307190858231333, 'верно'], ['(2; 3)', 1.1029411373136861, 2.4307190858231333, 'верно']]
[[0.9345180276555175, '0.06', 'верно']]
[['(1; 2)', 0.7137521351726992, '0.06', 'верно'], ['(1; 3)', 0.8758447617893016, '0.06', 'верно'], ['(2; 3)', 0.8330857713105322, '0.06', 'верно']]


In [10]:
def stick(anal):#Sick of Truth
    g = True
    for i in anal:
        g = g and (i[len(i)-1] == 'верно')
    return g

def anal_1_1_str(ql):
    s = {
        True : 'Гипотеза о равенстве математических ожиданий не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве математических ожиданий противоречит экспериментальным данным (не может быть принята) при уровне значимости  0,06.'
    }
    g = True
    N = len(ql[0])
    for i in range (3):
        for k in range(i):
            g = g and(abs(TNM(ql[k],ql[i])) <= stats.t.ppf(0.97,2*(N-1)))
    return [s[g]]

def anal_1_2_str(ql):
    m = len(ql)-1
    n = len(ql)*(ql[0].size-1)
    z = stats.f.ppf(0.94,m,n)
    f = FNM_1(ql)
    s1 = 'z_alpha = ' + str('%.5f'% z)
    s2 = 'F_NM = ' + str('%.5f'% f)
    s3 = {
        True : 'Гипотеза о равенстве математических ожиданий трёх случайных величин, выборки которых находятся в столбцах массива U={u_i,j|1<= i <= N,1 <= j <= 3}, не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве математических ожиданий трёх случайных величин, выборки которых находятся в столбцах массива U={u_i,j|1<= i <= N,1 <= j <= 3}, противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    return [s1,s2,s3[f<=z]]

def anal_1_3_str(ql):
    ao = anal_1_3(ql)
    s1 = 'pval = anova(U)= ' + str('%.5f'% ao[0][0])
    if ao[0][0] > 0.06:
        s1 = s1 + ' > '
    if ao[0][0] == 0.06:
        s1 = s1 + ' = '
    if ao[0][0] < 0.06:
        s1 = s1 + ' < '
    s1 = s1 + 'alpha = 0.06'
    s2 = {
        True : 'Гипотеза о равенстве математических ожиданий трёх случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве математических ожиданий трёх случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    return [s1, s2[ao[0][0] > 0.06]]

def anal_1_4_str(ql):
    s = {
        True : 'Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False: 'Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    g = True
    for i in out_1_4(ql):
        g = g and (i[1] >= 0.06)
    return [s[g]]

def anal_1_5_str(ql):
    s = {
        True : 'Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False: 'Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    g = True
    for i in out_1_5(ql):
        g = g and (i[1] >= 0.06)
    return [s[g]]    

def anal_2_1_str(ql):
    s = {
        True : 'Гипотеза о равенстве дисперсий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве дисперсий соответствующих пар случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    g = True
    for i in anal_2_1(ql):
        g = g and (i[3] == 'верно')
    return [s[g]]
    
def anal_2_2_str(ql):
    ao = anal_2_2(ql)
    s1 = 'pval = bartlett_test (X, Y, Z) = ' + str('%.5f'% ao[0][0])
    if ao[0][0] > 0.06:
        s1 = s1 + ' > '
    if ao[0][0] == 0.06:
        s1 = s1 + ' = '
    if ao[0][0] < 0.06:
        s1 = s1 + ' < '
    s1 = s1 + '0.06'
    s2 = {
        True : 'Гипотеза о равенстве дисперсий трёх случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве дисперсий трёх случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    return [s1,s2[ao[0][2]=='верно']]

def anal_2_3_str(ql):
    s = {
        True : 'Гипотеза о равенстве дисперсий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.',
        False : 'Гипотеза о равенстве дисперсий соответствующих пар случайных величин противоречит экспериментальным данным (не может быть принята) при уровне значимости alpha = 0,06.'
    }
    g = True
    for i in anal_2_3(ql):
        g = g and (i[3]=='верно')
    return [s[g]]

print(anal_1_1_str(ql))
print(anal_1_2_str(ql))
print(anal_1_3_str(ql))
print(anal_1_4_str(ql))
print(anal_1_5_str(ql))
print(anal_2_1_str(ql))
print(anal_2_2_str(ql))
print(anal_2_3_str(ql))
print()

def out_1_2_str(ql):
    o = out_1_2(ql)
    l = []
    for i in range(len(o)):
        s = head_1_2_str[i] + ' = ' 
        if type(o[i]) != int:
            s = s + str('%.5f'% o[i])
        else:
            s = s + str(o[i])
        l.append(s)
    return l

for i in out_1_2_str(ql):
    print(i)

['Гипотеза о равенстве математических ожиданий не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.']
['z_alpha = 2.95696', 'F_NM = 0.68177', 'Гипотеза о равенстве математических ожиданий трёх случайных величин, выборки которых находятся в столбцах массива U={u_i,j|1<= i <= N,1 <= j <= 3}, не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.']
['pval = anova(U)= 0.38469 > alpha = 0.06', 'Гипотеза о равенстве математических ожиданий трёх случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.']
['Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимости alpha = 0,06.']
['Гипотеза о равенстве математических ожиданий соответствующих пар случайных величин не противоречит экспериментальным данным (может быть принята) при уровне значимос

In [11]:
def strm(a):
    n = len(a)
    b = a[0]
    m = len(b)
    for i in range(n):
        for k in range(m):
            if type(a[i][k]) == int:
                a[i][k] = str(a[i][k])
            elif type(a[i][k]) != str:
                a[i][k] = str('%.5f'% a[i][k])

def tabler(document, out, head = None):
    tl = []
    if head != None:
        tl.append(head)
    for i in out:
        tl.append(i)
    strm(tl)
    
    table = document.add_table(rows = len(tl),cols = len(tl[0]))
    for i in range(len(tl)):
        hdr_cells = table.rows[i].cells
        for k in range(len(tl[0])):
            if type(tl[i][k]) == str:
                hdr_cells[k].text = tl[i][k]
            else:
                pass #LaTeX

def stringer(document, strs):
    for i in strs:
        document.add_paragraph(i)

def docker(ql, filename):
    document = Document()
    document.add_paragraph('Результаты расчетов')
    
    document.add_paragraph('Данный двумерный массив:')
    qh = [[ql[i][k] for i in range(len(ql))] for k in range(len(ql[0]))]
    tabler(document,qh)
    document.add_paragraph('')

    document.add_paragraph('Задание 1: гипотеза о равенстве математических ожиданий')
    document.add_paragraph('Пункт 1)')
    tabler(document, out_1_1(ql), head_1_1_str)
    document.add_paragraph('')
    document.add_paragraph('Пункт 2)')
    #tabler(document, [out_1_2(ql)], head_1_2_str)
    stringer(document, out_1_2_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 3)')
    document.add_paragraph('pval = anova(U) = ' + str('%.5f'% out_1_3(ql)))
    document.add_paragraph('')
    document.add_paragraph('Пункт 4)')
    tabler(document, out_1_4(ql), head_1_4_str)
    document.add_paragraph('')
    document.add_paragraph('Пункт 5)')
    tabler(document, out_1_5(ql), head_1_5_str)
    document.add_paragraph('')
    document.add_paragraph('')
    document.add_paragraph('')
    
    
    document.add_paragraph('Задание 2: гипотеза о равенстве дисперсий')
    document.add_paragraph('Пункт 1)')
    tabler(document, out_2_1(ql), head_2_1_str)
    document.add_paragraph('')
    document.add_paragraph('Пункт 2)')
    document.add_paragraph('pval = bartlett_test (X, Y, Z ) =' + str('%.5f'% out_2_2(ql)))
    document.add_paragraph('')
    document.add_paragraph('Пункт 3)')
    tabler(document, out_2_3(ql), head_2_3_str)
    document.add_paragraph('')
    document.add_paragraph('')
    document.add_paragraph('')
    document.add_paragraph('')
    document.add_paragraph('')
    
    document.add_paragraph('Анализ результатов и выводы:')
    document.add_paragraph('Задание 1')
    document.add_paragraph('Пункт 1)')
    tabler(document,anal_1_1(ql),anal_head_1_1_str)
    stringer(document,anal_1_1_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 2)')
    tabler(document,anal_1_2(ql),anal_head_1_2_str)####
    stringer(document,anal_1_2_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 3)')
    tabler(document,anal_1_3(ql),anal_head_1_3_str)####
    stringer(document,anal_1_3_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 4)')
    tabler(document,anal_1_4(ql),anal_head_1_4_str)
    stringer(document,anal_1_4_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 5)')
    tabler(document,anal_1_5(ql),anal_head_1_5_str)
    stringer(document,anal_1_5_str(ql))
    document.add_paragraph('')
    document.add_paragraph('')
    document.add_paragraph('')
    
    document.add_paragraph('Задание 2')
    document.add_paragraph('Пункт 1)')
    tabler(document,anal_2_1(ql),anal_head_2_1_str)
    stringer(document,anal_2_1_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 2)')
    tabler(document,anal_2_2(ql),anal_head_2_2_str)####
    stringer(document,anal_2_2_str(ql))
    document.add_paragraph('')
    document.add_paragraph('Пункт 3)')
    tabler(document,anal_2_3(ql),anal_head_2_3_str)
    stringer(document,anal_2_3_str(ql))
    document.add_paragraph('')
    
    document.save(filename + '.docx')
    
    
    print('Done!')

In [12]:
docker(ql, 'Data/result')

Done!
